# Import csv for book URLs and get all titles

In [ ]:
import pandas as pd
# read the csv file for book urls
title_all = pd.read_csv("BooksURLs.csv")
title_add= list(title_all.newUrl)
len(title_add)

# Get all VIAF/DNB links

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import Lib.urllib.request
import rdflib
from itertools import islice
import requests
import glob

from rdflib import Graph

In [ ]:
# get the name VIAF/DNB urls
viaf_unique = []
names_has_viaf = []
names_no_viaf = []
viaf_else = []
n = 0
d = {}
for i in title_add:
    d[i] = {}
    page = requests.get(i)
    soup = BeautifulSoup(page.content,'xml')
    creators = soup.find_all('name')
    for m in creators:
        if 'valueURI' in m:
            names_has_viaf.append(m.namePart.text)
        else:
            try:
                if m['valueURI']:
                    names_has_viaf.append(m.namePart.text)
            except:
                if 'type' in m:
                    if m['type'] == "personal":
                        names_no_viaf.append(m.namePart.text)
                else:
                    n+=1
                    try:
                        viaf_else.append(m.namePart.text)
                    except:
                        viaf_else.append(m)
        try:
            d[i][str(m['valueURI'].rstrip("/"))] = {}
            if str(m['valueURI'].rstrip("/")) not in viaf_unique:
                viaf_unique.append(m['valueURI'].rstrip("/"))
        except:
            pass

In [ ]:
# get the number of names with viaf urls and without viaf urls
print(len(names_has_viaf))
print(len(set(names_no_viaf)))

In [ ]:
# get the number of names with unique viaf/dnb links
all_names = []
names = []
for link in d.keys():
    for viaf in d[link]:
        if viaf not in names:
            all_names.append(d[link][viaf])
            names.append(viaf)
        else:
            pass
print(len(all_names))

In [ ]:
# get the number of names with unique dnb links
dnb_viaf = []
n = 0
for i in viaf_unique:
    if not i.startswith("http://viaf.org/viaf/") and i not in dnb_viaf:
        dnb_viaf.append(i)
    else:
        n+=1
print(len(dnb_viaf))

In [ ]:
# get the number of names with unique viaf links
origin_viaf = []
for i in viaf_unique:
    if i.startswith("http://viaf.org/viaf/") and i not in dnb_viaf:
        origin_viaf.append(i.rstrip("/"))
print(len(origin_viaf))

# Covert DNB IDs to VIAF IDs 

In [ ]:
# Cover DNB IDs to VIAF IDs using VIAF Translate Source API
new_viaf = {}
failed_covert_dnb_viaf = []
n = 0
for link in dnb_viaf:
    dnb_id = link.strip("http://d-nb.info/gnd/")
    dnb_html = "http://www.viaf.org/viaf/sourceID/DNB%7c" + dnb_id
    r = requests.get(dnb_html)
    soup = BeautifulSoup(r.content, 'html.parser')
    print(n)
    n += 1
    try:
        viaf_id = soup.findAll("title")[0].text
        viaf_link = "http://viaf.org/viaf/" + viaf_id
        if viaf_id != "Document not found":
            if link not in new_viaf:
                new_viaf[link] = viaf_link
            if link in new_viaf and viaf_link != new_viaf[link]:
                print(new_viaf[link], viaf_link)
        else:
            failed_covert_dnb_viaf.append(link)
    except:
        pass

In [ ]:
# get the number of duplicated viaf IDs after conversion
n = 0
for v in new_viaf.values():
    if v in origin_viaf:
        n+=1

In [ ]:
# get the number of DNB IDs that cannot be mapped to VIAF
all_link_after_covert = []
d1 = {}
for key in d:
    d1[key] = {}
    for kk in d[key]:
        if kk not in all_link_after_covert:
            all_link_after_covert.append(kk)
        if kk in new_viaf:
            d1[key][new_viaf[kk]] = {}
        else:
            kk = kk.rstrip("/")
            d1[key][kk] = {}
            
n = 0
no_viaf_dnb_unique = []
for k in d1:
    for kk in d1[k]:
        if not kk.startswith("http://viaf.org/viaf/") and kk not in no_viaf_dnb_unique:
            n+=1
            no_viaf_dnb_unique.append(kk)

len(no_viaf_dnb_unique)

# Find all name authorities + remove viaf IDs that have no name authorities

In [ ]:
d2 = {}
failed_links = [] #cannot open viaf links
for link in d1.keys():
    d2[link]={}
    for viaf in d1[link].keys():
        d2[link][viaf] = {}
        if viaf.startswith("http://viaf.org/viaf/"):
            l = viaf.rstrip("/") + "/justlinks.json" 
            r = requests.get(l)
            try:
                json = r.json()
                if "WKP" in json and json["WKP"] != []:
                    wiki = json["WKP"]
                    d2[link][viaf]["wiki"] = {}
                    d2[link][viaf]["wiki"][wiki[0]]=[]
                if "BNF" in json and json["BNF"] != []:
                    bnf = json["BNF"]
                    d2[link][viaf]["bnf"]= {}
                    d2[link][viaf]["bnf"][bnf[0]]=[]
                if "DNB" in json and json["DNB"]!=[]:
                    dnb = json["DNB"]
                    d2[link][viaf]["dnb"] = {}
                    d2[link][viaf]["dnb"][dnb[0]] = []
                if "LC" in json and json["LC"] !=[]:
                    lc = json["LC"]
                    d2[link][viaf]["lc"] = {}
                    if lc[0] != "LNB:BSLC;=BA":
                        d2[link][viaf]["lc"][lc[0]] = []
                    else:
                        print(lc)
                        d2[link][viaf]["lc"][lc[1]] = []
                if "JPG" in json and json["JPG"]!=[]:
                    getty = json["JPG"]
                    d2[link][viaf]["getty"] = {}
                    d2[link][viaf]["getty"][getty[0]] = []
            except:
                if (link,viaf) not in failed_links:
                    failed_links.append((link, viaf))
                else:
                    pass
        else:
            d2[link][viaf]={}
            d2[link][viaf]["dnb"]= {}
            d2[link][viaf]["dnb"][viaf] = []
            print(d2[link][viaf])

In [ ]:
# get the number of names that exits in at least one of the selected authorities
all_names2 = []
names2 = []

for link in d2.keys():
    for viaf in d2[link]:
        if viaf not in names2:
            all_names2.append(d2[link][viaf])
            names2.append(viaf)
        else:
            pass
print(len(all_names2))

In [ ]:
# write the dictionary to a csv file
with open("results1.json", "r", encoding='utf-8') as read_file:
    d2 = json.load(read_file)

In [ ]:
# get the number of obsolete VIAF IDs
varify_failed = []
for link in d2:
    for viaf in d2[link]:
        if d2[link][viaf] == {}:
            if (link, viaf) not in failed_links:
                print((link, viaf))
            if (link,viaf) not in varify_failed:
                varify_failed.append((link,viaf))
no_viaf = [i[1] for i in varify_failed if i not in no_viaf]
no_viaf = list(set(no_viaf))
print(len(no_viaf))

In [ ]:
# manually find the IDs of selected authorities for names that are available in DNB but not in VIAF

# Count authorities

In [ ]:
import json
with open("results2.json", "r", encoding='utf-8') as read_file:
    d4 = json.load(read_file)
d = d4.copy()

In [ ]:
# get the number of person name entities that are available in the selected authorities
c_dnb = []
dnb_unique = []
c_wiki = []
wiki_unique = []
c_getty = []
getty_unique = []
c_lc = []
lc_unique = []
c_bnf = []
bnf_unique = []
for title in d:
    for viaf in d[title]:
        for aut in d[title][viaf]:
            if aut == "wiki" and viaf not in c_wiki:
                c_wiki.append(viaf)
                wiki_unique.append(list(d[title][viaf]["wiki"].keys())[0])
            if aut == "dnb" and viaf not in c_dnb:
                c_dnb.append(viaf)
                dnb_unique.append(list(d[title][viaf]["dnb"].keys())[0])
            if aut == "bnf" and viaf not in c_bnf:
                c_bnf.append(viaf)
                bnf_unique.append(list(d[title][viaf]["bnf"].keys())[0])
            if aut == "lc" and viaf not in c_lc:
                c_lc.append(viaf)
                lc_unique.append(list(d[title][viaf]["lc"].keys())[0])
            if aut == "getty" and viaf not in c_getty:
                c_getty.append(viaf)
                getty_unique.append(list(d[title][viaf]["getty"].keys())[0])
                
print("wiki ", len(c_wiki))
print("getty ",len(c_getty))
print("lc ", len(c_lc))
print("bnf ", len(c_bnf))
print("dnb ", len(c_dnb))

# Get property names for name entities in DNB

In [ ]:
d_dnb = {}
error = []
not_processed = []
n = 0
for dnb_id in dnb_unique:
    print(n)
    n += 1
    renewed_url = 'https://hub.culturegraph.org/entityfacts/' + dnb_id.lstrip("https://d-nb.info/gnd/")
    print(renewed_url)
    try:
        page = requests.get(renewed_url)
        soup = BeautifulSoup(page.content, 'html.parser')
        newDictionary=json.loads(str(soup))
        r = list(newDictionary.keys())
        print(r)
        if r != ['Error']:
            d_dnb[dnb_id] = r
        else:
            error.append(dnb_id)
    except:
        not_processed.append(dnb_id)

In [ ]:
# create a data frame for properties
no_dnb = []
d1 = {}
for link in d:
    d1[link] = {}
    for viaf in d[link]:
        d1[link][viaf] = {}
        for k in d[link][viaf]:
            if k == "dnb":
                key = list(d[link][viaf][k].keys())[0]
                if key in d_dnb:
                    d1[link][viaf]["dnb"] = {}
                    d1[link][viaf]["dnb"][key] = d_dnb[key]
                else:
                    print((link,viaf,key))
                    no_dnb.append((link,viaf,key))
            if k =="lc":
                key = list(d[link][viaf][k].keys())[0]
                d1[link][viaf]["lc"] = {}
                d1[link][viaf]["lc"][key] = []
            if k =="getty":
                key = list(d[link][viaf][k].keys())[0]
                d1[link][viaf]["getty"] = {}
                d1[link][viaf]["getty"][key] = []
            if k =="bnf":
                key = list(d[link][viaf][k].keys())[0]
                d1[link][viaf]["bnf"] = {}
                d1[link][viaf]["bnf"][key] = []
            if k =="wiki":
                key = list(d[link][viaf][k].keys())[0]
                d1[link][viaf]["wiki"] = {}
                d1[link][viaf]["wiki"][key] = []

# Get property names for name entities in wikidata

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [ ]:
remove_id = ["Repertorium van ambtsdragers en ambtenaren id",'PSS-Archi architect id',  'EAGLE id',  'BALaT person/organisation id',  'NMVW id',  'National Library of Korea Identifier',  'Swedish Open Cultural Heritage URI', "Libris-URI", "J. Paul Getty Museum artist id", "Royal Academy new identifier", "Image Archive, Herder Institute", "Europeana entity"]

In [ ]:
d_wiki = {}    
n = 0
cannot = []
for key in wiki_unique:
    w_link = w_link = '<http://www.wikidata.org/entity/' + key +">"
    try:
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        # From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
        sparql.setQuery("""
        SELECT DISTINCT ?p ?pLabel ?propLabel
        WHERE 
        {
           %s  ?p ?o.

          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". } 
          ?prop wikibase:directClaim ?p .
        }

        """%w_link)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        results_df = pd.io.json.json_normalize(results['results']['bindings'])
        r = [i for i in list(results_df["propLabel.value"]) if "ID" not in i]
        d_wiki[key] = r
    except:
        cannot.append(key)
    print(n)
    n+=1

In [ ]:
for link in d1:
    for viaf in d1[link]:
        for k in d1[link][viaf]:
            if k == "wiki":
                wiki_id = list(d1[link][viaf][k].keys())[0]
                d1[link][viaf][k][wiki_id] = new_wiki_d[wiki_id]

# Get property names for name entities in BNF

In [ ]:
import re
d_bnf = {}
bnf_problem = []
n1= 0
for key in bnf_unique:
    if key[:3] == 'FRB':
        bnf_link = "https://data.bnf.fr/en/" + key[5:-1]
    else:
        bnf_link = "https://data.bnf.fr/en/" + key[-9:-1]
    try:
        page = requests.get(bnf_link)
        soup = BeautifulSoup(page.content, 'html.parser')
        n = 0
        r = []
        for row in soup.find_all('td'):
            if n % 2 == 0:
                r.append(row.text.strip().strip("\xa0:"))
            n += 1
        r2 = []
        h3 = soup.find_all("h3")
        for i in h3:
            r2.append(re.sub(r'\([^)]*\)', '', i.text.strip("\xa0:").strip().replace(" ","").replace("\n","")))
        r += r2
        print(r)
        d_bnf[key] = r
    except:
        bnf_problem.append(key)
    n1+=1

In [ ]:
for link in d1:
    for viaf in d1[link]:
        for k in d1[link][viaf]:
            if k == "bnf":
                bnf_id = list(d1[link][viaf][k].keys())[0]
                d1[link][viaf][k][bnf_id] = d_bnf[bnf_id]

# Get property names for name entities in Getty

In [ ]:
basic = ['Note',
 'Names',
 'preferred',
 'Nationalities',
 'preferred',
 'Roles',
 'preferred',
 'Gender',
 'Birth and Death Places',
 'List/Hierarchical Position',
 'Related People or Corporate Bodies',
 'Biographies',
 'Additional Names',
 'Sources and Contributors',
 'Subject',
'Events', 'Note']
d_getty = {}
for key in getty_unique:
    print("For Getty", key)
    g_link = 'http://vocab.getty.edu/page/ulan/' + key
    page = requests.get(g_link)
    soup = BeautifulSoup(page.content, 'html.parser')
    a = soup.find_all("span", {"class": "page"})
    r = []
    for i in a:
        b = i.find_all("b")
        for m in b:
            r.append(m.text.strip().strip(':'))   
    dump = list(set(r) - set(basic))
    r = [i for i in r if i not in dump]
    r1 = []
    for i in range(len(r)):
        if i > 0:
            if r[i-1] == "Names" and r[i] == 'preferred':
                r1.append("Names preferred")
            elif r[i-1] == "Nationalities" and r[i] == 'preferred':
                r1.append("Nationalities preferred") 
            elif r[i-1] == "Roles" and r[i] == 'preferred':
                r1.append("Roles preferred") 
            else:
                r1.append(r[i])
        else:
            r1.append(r[i])
    d_getty[key]= r1

In [ ]:
for link in d1:
    for viaf in d1[link]:
        for k in d1[link][viaf]:
            if k == "getty":
                g_id = list(d1[link][viaf][k].keys())[0]
                d1[link][viaf][k][g_id] = d_getty[g_id]

# Get property names for name entities in LC

In [ ]:
d_lc = {}
for key in lc_unique:
    lc_link1 = "http://id.loc.gov/authorities/names/" + key
    page = requests.get(lc_link1)
    soup = BeautifulSoup(page.content, 'html.parser')
    r = soup.findAll("h3")
    r1 = soup.findAll("dt", {"title": "bf:Title"})
    r += r1
    r2 = []
    for node in r:
        r2.append(''.join(node.findAll(text=True)))
    print(r2)
    d_lc[key] = r2

In [ ]:
for link in d1:
    for viaf in d1[link]:
        for k in d1[link][viaf]:
            if k == "lc":
                lc_id = list(d1[link][viaf][k].keys())[0]
                d1[link][viaf][k][lc_id] = d_lc[lc_id]

# Export the data frame to a csv file

In [ ]:
df1 = pd.DataFrame(columns=["book_ind","link", "viaf", "dnb","lc", "wiki","bnf","getty"])
d = d1.copy()
n = 0
for link in d:
    n+=1
    for viaf in d[link]:
        l = [link, viaf]
        if "wiki" in d[link][viaf]:
            key = list(d[link][viaf]['wiki'].keys())[0]
            l.append(d[link][viaf]['wiki'][key])
        else:
            l.append("")
        if "dnb" in d[link][viaf]:
            key = list(d[link][viaf]['dnb'].keys())[0]
            l.append(d[link][viaf]['dnb'][key])
        else:
            l.append("")
        if "bnf" in d[link][viaf]:
            key = list(d[link][viaf]['bnf'].keys())[0]
            l.append(d[link][viaf]['bnf'][key])
        else:
            l.append("")
        if "getty" in d[link][viaf]:
            key = list(d[link][viaf]['getty'].keys())[0]
            l.append(d[link][viaf]['getty'][key])
        else:
            l.append("")
        if "lc" in d[link][viaf]:
            key = list(d[link][viaf]['lc'].keys())[0]
            l.append(d[link][viaf]['lc'][key])
        else:
            l.append("")
        df1 = df1.append({"book_ind": n, "link": link, "viaf": viaf,"wiki": l[2],"dnb": l[3],"bnf": l[4],"getty": l[5],"lc":l[6]}, ignore_index=True)

In [ ]:
df1.to_csv("results3.csv")

# Get the number of properties for name entities in DNB

In [ ]:
dnb_v1 = df1['dnb']
dnb_unique1 = []
for i in dnb_v1:
    for m in i:
        if m not in dnb_unique1:
            dnb_unique1.append(m)

In [ ]:
d_dnb1 = {}
for i in dnb_unique1:
    d_dnb1[i] = 0
count = 0
for name in all_names:
    if "dnb" in name:
        count+=1
        d_d1 = name['dnb']
        dnb_key1 = list(d_d1.keys())[0]
        dnb_list1 = d_d1[dnb_key1]
        print(dnb_list1)
        for i in dnb_list1:
            d_dnb1[i] += 1

# Get the number of properties for name entities in LC

In [ ]:
lc_v1 = df1['lc']
lc_unique1 = []
for i in lc_v1:
    for m in i:
        if m not in lc_unique1:
            lc_unique1.append(m)
d_lc1 = {}
for i in lc_unique1:
    d_lc1[i] = 0
count = 0
for name in all_names:
    if "lc" in name:
        count+=1
        d_d1 = name['lc']
        lc_key1 = list(d_d1.keys())[0]
        lc_list1 = d_d1[lc_key1]
        print(lc_list1)
        for i in lc_list1:
            d_lc1[i] += 1

# Get the number of properties for name entities in wikidata

In [ ]:
wiki_v1 = df1['wiki']
wiki_unique1 = []
for i in wiki_v1:
    for m in i:
        if m not in wiki_unique1:
            wiki_unique1.append(m)
d_wiki1 = {}
for i in wiki_unique1:
    d_wiki1[i] = 0
count = 0
for name in all_names:
    if "wiki" in name:
        count+=1
        d_d1 = name['wiki']
        wiki_key1 = list(d_d1.keys())[0]
        wiki_list1 = d_d1[wiki_key1]
        print(wiki_list1)
        for i in wiki_list1:
            d_wiki1[i] += 1

# Get the number of properties for name entities in BNF

In [ ]:
bnf_v1 = df1['bnf']
bnf_unique1 = []
for i in bnf_v1:
    for m in i:
        if m not in bnf_unique1:
            bnf_unique1.append(m)
d_bnf1 = {}
for i in bnf_unique1:
    d_bnf1[i] = 0
count = 0
for name in all_names:
    if "bnf" in name:
        count+=1
        d_d1 = name['bnf']
        bnf_key1 = list(d_d1.keys())[0]
        bnf_list1 = d_d1[bnf_key1]
        print(bnf_list1)
        for i in bnf_list1:
            d_bnf1[i] += 1

# Get the number of properties for name entities in Getty

In [ ]:
getty_v1 = df1['getty']
getty_unique1 = []
for i in getty_v1:
    for m in i:
        if m not in getty_unique1:
            getty_unique1.append(m)
d_getty1 = {}
for i in getty_unique1:
    d_getty1[i] = 0
count = 0
for name in all_names:
    if "getty" in name:
        count+=1
        d_d1 = name['getty']
        getty_key1 = list(d_d1.keys())[0]
        getty_list1 = d_d1[getty_key1]
        print(getty_list1)
        for i in getty_list1:
            d_getty1[i] += 1

# Compare BnF JSON with JSONLD

In [ ]:
all_bnf = []
for key in bnf_unique:
    if key[:3] == 'FRB':
        bnf_link = "https://data.bnf.fr/en/" + key[5:-1]
    else:
        bnf_link = "https://data.bnf.fr/en/" + key[-9:-1]
    all_bnf.append(bnf_link)

In [ ]:
all_jsonld = []
all_json = []
for i in all_bnf:
    page = requests.get(i)
    prefix = page.url
    ld = prefix + "rdf.jsonld"
    js = prefix.rstrip("/") + ".json"
    all_jsonld.append(ld)
    all_json.append(js)

In [ ]:
# jsonld
def parse_jsonld_1(mydata, d):
    for child in mydata['@graph']:
        if "dcterms:creator" in child:
            if "works" not in d:
                d['works'] = [child["@id"].replace("#about", "")]
            else:
                d['works'].append(child["@id"].replace("#about", ""))
            
        if '#about' in child["@id"]:
            aboutauthor = child

            # gender
            if 'foaf:gender' in aboutauthor:
                d["gender"] = 'foaf:gender'

            # image
            if 'foaf:depiction' in aboutauthor:
                d["image"] = 'foaf:depiction'

            # birth_date
            for bd in birth_date_pool:
                if bd in aboutauthor:
                    if 'birthdate' not in d:
                        d["birthdate"] = [bd]
                    else:
                        d["birthdate"].append(bd)

            # death_date
            for dd in death_date_pool:
                if dd in aboutauthor:
                    if 'deathdate' not in d:
                        d["deathdate"] = [dd]
                    else:
                        d["deathdate"].append(dd)

            # birth_place
            for bp in birth_place_pool:
                if bp in aboutauthor:
                    if 'birthplace' not in d:
                        d["birthplace"] = [bp]
                    else:
                        d["birthplace"].append(bp)

            # death_place
            for dp in death_place_pool:
                if dp in aboutauthor:
                    if 'deathplace' not in d:
                        d["deathplace"] = [dp]
                    else:
                        d["deathplace"].append(dp)

             # language
            for lan in language_pool:
                if lan in aboutauthor:
                    if 'language' not in d:
                        d["language"] = [lan]
                    else:
                        d["language"].append(lan)

            # biographical
            for bio in biographicalInformation_pool:
                if bio in aboutauthor:
                    if 'notes' not in d:
                        d['notes'] = [bio]
                    else:
                        d['notes'].append(bio) 
            
            # nationality
            for nation in nationality_pool:
                if nation in aboutauthor:
                    if 'nationality' not in d:
                        d["nationality"] = [nation]
                    else:
                        d["nationality"
                         ].append(nation)
            

        if "#" not in child["@id"]:
            generalauthor = child
            # variant names
            if 'skos:altLabel' in generalauthor:
                if "variant_names" not in d:
                    d["variant_names"] = "skos:altLabel"
                    print("variant name")

            # biographical
            if 'skos:note' in generalauthor:
                if 'notes' not in d:
                    d['notes'] = ["skos:note"]
                else:
                    d['notes'].append("skos:note") 

                print("note")

        if "#Expression" in child["@id"]:
            if "contributors" not in d: 
                d["contributors"] = [child]
            else:
                d["contributors"].append(child)
    return d

_int = 0
d_author = {}
for uri in all_jsonld:
    print(_int)
    key = uri.replace('/rdf.jsonld', "")
    # The content type you want to set in the Request Headers.
    # This example is for RDF/XML
    request_headers = {'Accept': 'application/rdf+xml'}
    # Build the request with the URI and Header parameters 
    request = urllib2.Request(uri, headers = request_headers)
    # Fetch the request
    response = urllib2.urlopen(request)
    # Read and Print the request
    data1 = response.read()
    dict_str = data1.decode("UTF-8")
    mydata = ast.literal_eval(dict_str)
    d = {}
    d = parse_jsonld_1(mydata,d)
    d_author[key] = d
    _int += 1

In [ ]:
# json
unwanted = ['name','firstname','site_name','type','isni','ark','modification-date','url','surname','stopdate','startdate','notice-type','title','label']
c_notes= 0
c_works = 0
d_json = {}
for i in all_json:
    r = requests.get(i)
    js = r.json()
    js_keys = list(js[0].keys())
    want = list(set(js_keys)-set(unwanted))
    if len(js[0]['works']) == 0:
        want.remove("works")
        c_works += 1
    if len(js[0]['notes']) == 0:
        want.remove("notes")
        c_notes+=1
    j_key = i.replace('.json', "")
    d_json[j_key] = want

In [2]:
# compare json with jsonld

In [ ]:
result = {}
for key in (d_json.keys() | d_jsonld.keys()):
    if key in d_json: result.setdefault(key, []).append(d_json[key])
    if key in d_jsonld: result.setdefault(key, []).append(d_jsonld[key])

In [ ]:
df = pd.DataFrame.from_dict(result, orient='index', columns=['JSON', 'JSONLD'])

In [ ]:
# compare keys
jsonld_image = []
jsonld_notes = []
json_birthdate = []
json_deathdate = []
category = []
for k in d_error.keys():
    for pair in d_error[k]:
        if pair == ('jsonld_does_not_have', 'image'):
            jsonld_image.append(k)
            
        elif pair == ('jsonld_does_not_have', 'notes'):
            jsonld_notes.append(k)
        
        elif pair == ('json_does_not_have', 'birthdate'):
            json_birthdate.append(k)
            
        else:
            json_deathdate.append(k)

In [ ]:
# get the value of roles
all_roles = []
for url in d_jsonld:
    if 'works' in d_jsonld[url]:
        for i in d_jsonld[url]['works']:
            for k in i:
                if 'bnfroles' in k:
                    m = 'https://data.bnf.fr/en/vocabulary/roles/' + k.split(":")[1]
                    if m not in all_roles:
                        all_roles.append(m)
dic_roles = {}
for url in all_roles:
    if url not in dic_roles:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        role_string = soup.findAll("title")[0].text
        dic_roles[url] = role_string

In [ ]:
# count the number of properties
birth_date_pool = ['bio:birth','rdaa:P50121','rdagroup2elements:dateOfBirth','bnf-onto:firstYear']
death_date_pool = ['bio:death','rdaa:P50120', 'rdagroup2elements:dateOfDeath', 'bnf-onto:lastYear']
birth_place_pool = ['rdagroup2elements:placeOfBirth', 'rdaa:P50119']
death_place_pool = ['rdagroup2elements:placeOfDeath', 'rdaa:P50118']
language_pool = ['rdagroup2elements:languageOfThePerson','rdaa:P50102' ]
biographicalInformation_pool = ['rdaa:P50113', 'rdagroup2elements:biographicalInformation']
nationality_pool = ["rdaa:P50097", "rdagroup2elements:countryAssociatedWithThePerson"]

d_birthdate = {'bio:birth': 0,'rdaa:P50121': 0,'rdagroup2elements:dateOfBirth':0,'bnf-onto:firstYear':0}
d_deathdate = {'bio:death':0 ,'rdaa:P50120':0, 'rdagroup2elements:dateOfDeath':0, 'bnf-onto:lastYear':0}
d_birthplace = {'rdagroup2elements:placeOfBirth' : 0, 'rdaa:P50119':0}
d_deathplace = {'rdagroup2elements:placeOfDeath':0, 'rdaa:P50118':0}
d_language= {'rdagroup2elements:languageOfThePerson': 0,'rdaa:P50102':0}   
d_bio = {'rdaa:P50113':0, 'rdagroup2elements:biographicalInformation':0, "skos:note":0}
d_nation = {"rdaa:P50097":0, "rdagroup2elements:countryAssociatedWithThePerson":0}

In [ ]:
for v in d_jsonld.values():
    if "notes" in v:
        for a in v["notes"]:
            d_bio[a] += 1
    if "birthdate" in v:
        for b in v["birthdate"]:
            d_birthdate[b] += 1
    if "deathdate" in v:
        for c in v["deathdate"]:
            d_deathdate[c] += 1
    if "birthplace" in v:
        for e in v["birthplace"]:
            d_birthplace[e] += 1
    if "deathplace" in v:
        for f in v["deathplace"]:
            d_deathplace[f] += 1
    if "language" in v:
        for d in v["language"]:
            d_language[d] += 1
    if "nationality" in v:
        for g in v["nationality"]:
            d_nation[g] += 1  

In [ ]:
no = []
for i in d_json:
    r = requests.get(i+".json")
    js = r.json()
    if 'notes' in js[0]:
        no.append((i,js[0]["notes"]))

In [ ]:
c_bd = 0
c_dd = 0
for i in d_json:
    if "birthdate" in d_json[i]:
        c_bd +=1
    if "deathdate" in d_json[i]:
        c_dd +=1
c_bd1 = 0
c_dd1 = 0
for i in d_jsonld:
    if "birthdate" in d_jsonld[i]:
        c_bd1 +=1
    if "deathdate" in d_jsonld[i]:
        c_dd1 +=1

c_vn = 0
c_ge = 0
c_na = 0
for i in d_jsonld:
    if "variant_names" in d_jsonld[i]:
        c_vn +=1
    if "gender" in d_jsonld[i]:
        c_ge +=1
    if "nationality" in d_jsonld[i]:
        c_na +=1
c_bp = 0
c_dp = 0
for i in d_jsonld:
    if "birthplace" in d_jsonld[i]:
        c_bp +=1
    if "deathplace" in d_jsonld[i]:
        c_dp +=1

c_lan = 0
for i in d_jsonld:
    if "language" in d_jsonld[i]:
        c_lan +=1

c_ima = 0
for i in d_jsonld:
    if "image" in d_jsonld[i]:
        c_ima +=1
c_no = 0
for i in d_jsonld:
    if "notes" in d_jsonld[i]:
        c_no +=1
        

c_ima1 = 0
for i in d_json:
    if "image" in d_json[i]:
        c_ima1+=1

c_na1 = 0
for i in d_json:
    if "nationality" in d_json[i]:
        c_nc_no1 = 0

for i in d_json:
    if "notes" in d_json[i]:
        c_no1 +=1a1 +=1
        
